In [11]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when

In [5]:
def validate_data(df):
    """Функция для валидации загруженных данных"""
    print("\n=== Проверка качества данных ===")
    
    # 1. Проверка на пропущенные значения
    print("\nПропущенные значения:")
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()
    
    # 2. Проверка уникальности transaction_id
    print("\nДубликаты transaction_id:")
    df.groupBy("transaction_id").count() \
      .filter("count > 1") \
      .count() \
      .show()
    
    # 3. Проверка допустимых диапазонов значений
    print("\nНекорректные суммы транзакций:")
    df.filter(col("tx_amount") <= 0).count()
    
    # 4. Проверка временного диапазона
    print("\nВременной диапазон транзакций:")
    df.select(min("tx_datetime"), max("tx_datetime")).show()

In [6]:




if __name__ == "__main__":
    spark = SparkSession.builder.getOrCreate()
    
    try:
        # Загрузка данных
        df = spark.read.parquet("s3a://fraud-detection-data-otus-2025/parquet/*")
        
        # Проверка данных
        validate_data(df)
        
        # Сохранение проверенных данных
        df.write.mode("overwrite") \
           .parquet("s3a://fraud-detection-data-otus-2025/validated_data/")
        
    finally:
        spark.stop()

KeyboardInterrupt: 

In [ ]:
 spark.stop()

In [ ]:
# progress bar

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, min, max
import sys
import time

In [8]:
def print_progress(iteration, total, prefix='', suffix='', length=50, fill='█'):
    """
    Выводит прогресс-бар в консоль
    """
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    sys.stdout.write(f'\r{prefix} |{bar}| {percent}% {suffix}')
    sys.stdout.flush()
    if iteration == total:
        print()

In [9]:
def validate_data(df):
    """Функция для валидации загруженных данных"""
    print("\n=== Проверка качества данных ===")
    
    # 1. Проверка на пропущенные значения
    print("\n[1/4] Проверка пропущенных значений...")
    start_time = time.time()
    missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    missing_values.show(truncate=False)
    print(f"Проверка завершена за {time.time() - start_time:.2f} сек")
    
    # 2. Проверка уникальности transaction_id
    print("\n[2/4] Проверка дубликатов transaction_id...")
    start_time = time.time()
    duplicates_count = df.groupBy("transaction_id").count().filter("count > 1").count()
    print_progress(1, 1, prefix='Прогресс:', suffix='Завершено', length=50)
    print(f"Найдено дубликатов: {duplicates_count}")
    print(f"Проверка завершена за {time.time() - start_time:.2f} сек")
    
    # 3. Проверка допустимых диапазонов значений
    print("\n[3/4] Проверка некорректных сумм транзакций...")
    start_time = time.time()
    invalid_amounts = df.filter(col("tx_amount") <= 0).count()
    print_progress(1, 1, prefix='Прогресс:', suffix='Завершено', length=50)
    print(f"Найдено транзакций с неположительной суммой: {invalid_amounts}")
    print(f"Проверка завершена за {time.time() - start_time:.2f} сек")
    
    # 4. Проверка временного диапазона
    print("\n[4/4] Проверка временного диапазона транзакций...")
    start_time = time.time()
    time_range = df.select(min("tx_datetime"), max("tx_datetime"))
    print_progress(1, 1, prefix='Прогресс:', suffix='Завершено', length=50)
    time_range.show(truncate=False)
    print(f"Проверка завершена за {time.time() - start_time:.2f} сек")

In [10]:






if __name__ == "__main__":
    print("Инициализация Spark сессии...")
    spark = SparkSession.builder.getOrCreate()
    print("Spark сессия успешно создана\n")
    
    try:
        # Загрузка данных
        print("=== Загрузка данных ===")
        print("[1/2] Чтение данных из S3...")
        start_time = time.time()
        df = spark.read.parquet("s3a://fraud-detection-data-otus-2025/parquet/*")
        print_progress(1, 1, prefix='Прогресс:', suffix='Завершено', length=50)
        print(f"Загружено {df.count()} строк")
        print(f"Время загрузки: {time.time() - start_time:.2f} сек\n")
        
        # Проверка данных
        validate_data(df)
        
        # Сохранение проверенных данных
        print("\n=== Сохранение данных ===")
        print("[2/2] Запись проверенных данных в S3...")
        start_time = time.time()
        df.write.mode("overwrite").parquet("s3a://fraud-detection-data-otus-2025/validated_data/")
        print_progress(1, 1, prefix='Прогресс:', suffix='Завершено', length=50)
        print(f"Данные успешно сохранены")
        print(f"Время сохранения: {time.time() - start_time:.2f} сек")
        
    except Exception as e:
        print(f"\n!!! ОШИБКА: {str(e)}")
    finally:
        print("\nЗавершение Spark сессии...")
        spark.stop()
        print("Spark сессия остановлена")

Инициализация Spark сессии...
Spark сессия успешно создана

=== Загрузка данных ===
[1/2] Чтение данных из S3...
Прогресс: |██████████████████████████████████████████████████| 100.0% Завершено
Загружено 657923861 строк
Время загрузки: 16.43 сек


=== Проверка качества данных ===

[1/4] Проверка пропущенных значений...
+--------------+-----------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|transaction_id|tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+--------------+-----------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|14            |1365       |14         |17262      |14       |14             |14          |14      |14               |
+--------------+-----------+-----------+-----------+---------+---------------+------------+--------+-----------------+

Проверка завершена за 76.47 сек

[2/4] Проверка дубликатов transaction_id...
Прогрес

In [4]:
# время записи увеличено, т.к не хватило место, и спарк похоже оптимизировал процесс